# Shor's Algorithm Implementation for 12-bit Numbers

### Author: Adrian Huber (qe24m003@technikum-wien.at)

## 1. The Problem: Period Finding with Quantum Phase Estimation

A semiprime number N is the product of exactly two prime numbers p and q, and while multiplication of these primes is computationally trivial, the reverse process of factoring N into its prime constituents becomes exponentially harder as the number grows larger. This property, where both p and q are large prime numbers, forms the foundation of widely-used cryptographic systems like RSA, where the security relies on the practical impossibility of factoring large semiprimes using classical computers.

`N = p × q`
where p and q are prime numbers

Shor's algorithm solves the integer factorization problem by reducing it to a period-finding problem that can be solved efficiently on a quantum computer. For a 12-bit number N (like 3127) that we want to factor, and a randomly chosen coprime number a, we need to find the period r where:

`a^r mod N = 1`

This period r can then be used to find factors of N with high probability.

### The Quantum Part: Phase Estimation 

The core quantum component uses quantum phase estimation on the unitary operator:

`U|y⟩ ≡ |ay mod N⟩`

The algorithm has these key steps:
1. Initialize a superposition in counting register using Hadamard gates
2. Apply controlled modular multiplications 
3. Use inverse quantum Fourier transform
4. Measure to obtain phase s/r
5. Use classical post-processing (continued fractions) to find r

## 2. Implementation for 12-bit Numbers

Our implementation handles 12-bit semiprime numbers (like 3127 = 53 × 59). This requires:
- 12 qubits for counting register (phase estimation)
- 13 qubits for work register (modular arithmetic, ceil(log2(N)) + 1)
- Total circuit depth of about 781 gates

### imports

In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import math
from fractions import Fraction


# Semiprime numbers to factor
semiprimes = {
    1: 3127,  # 53 * 59 = 3127
    2: 3131  # 31 * 101 = 3131  (check if it wokrs for other prime numbers of same size)
}

### helper functions

In [2]:
def find_period(phase, N, a):
    """Estimates the period using continued fractions."""
    for i in range(1, 2000):  
        frac = Fraction(phase).limit_denominator(i)
        r = frac.denominator
        if (pow(a, r, N) == 1):
            return r
    return 0





def calculate_factors(N, a, r):
    """Classical post-processing to find factors."""
    if r % 2 != 0 or r == 0:
        return None, None
    
    x = pow(a, r // 2, N)
    factor1 = math.gcd(x - 1, N)
    factor2 = math.gcd(x + 1, N)
    
    if factor1 == 1 or factor2 == 1 or factor1 * factor2 != N:
        return None, None
    return factor1, factor2

### main function

In [3]:
def factor_semiprime(N, backend):
    """Main function implementing Shor's algorithm for 12-bit numbers."""
    # Find coprime a
    a = 2
    while math.gcd(a, N) != 1:
        a += 1
        if a > N:
            raise ValueError(f"No coprime 'a' found for N = {N}")
    
    # Circuit parameters
    n_count = 12  #12 counting qubits for 12-bit numbers
    target_size = math.ceil(math.log(N, 2)) + 1
    
    # Create quantum circuit
    qr = QuantumRegister(n_count + target_size, 'q')
    cr = ClassicalRegister(n_count, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Initialize counting register with Hadamard gates
    for q in range(n_count):
        qc.h(qr[q])
    
    # Initialize target register
    qc.x(qr[n_count])
    
    # Controlled modular exponentiation
    for q in range(n_count):
        power = 2**q
        c = pow(a, power, N)
        for i in range(target_size):
            if (c >> i) & 1:
                qc.cx(qr[q], qr[n_count + i])
    
    # Inverse QFT
    for j in range(n_count):
        for m in range(j):
            angle = -math.pi / float(2**(j-m))
            qc.cu1(angle, qr[m], qr[j])
        qc.h(qr[j])
    
    # Measure
    qc.measure([qr[i] for i in range(n_count)], cr)
    
    # Execute with increased shots for better accuracy
    job = backend.run(qc, shots=2048)  # Increased shots for larger numbers
    job_monitor(job)
    result = job.result()
    
    # Process results
    for measured_result in result.get_memory():
        phase = int(measured_result, 2) / (2**n_count)
        r = find_period(phase, N, a)
        if r > 0:
            factor1, factor2 = calculate_factors(N, a, r)
            if factor1 is not None:
                return factor1, factor2
    
    return None, None

### execute

In [4]:
def main():
    # Initialize Quantum Rings provider
    provider = QuantumRingsProvider()
    backend = provider.get_backend("scarlet_quantum_rings")
    
    # Factor each semiprime
    for bit_size, N in semiprimes.items():
        print(f"\nFactoring N = {N} (bit size 12)")
        factor1, factor2 = factor_semiprime(N, backend)
        
        if factor1 and factor2:
            print(f"Successfully factored {N} = {factor1} * {factor2}")
        else:
            print(f"Failed to factor {N}")

if __name__ == "__main__":
    main()


Factoring N = 3127 (bit size 12)
Job Queued
Job Running
Job Running
Job Done.
Ending Job Monitor
Successfully factored 3127 = 59 * 53

Factoring N = 3131 (bit size 12)
Job Queued
Job Running
Job Running
Job Done.
Ending Job Monitor
Successfully factored 3131 = 31 * 101


### Scalability Analysis
The implementation successfully factors 12-bit semiprime numbers but has a fundamental limitation in scaling to larger numbers. With hardcoded 12 qubits in the counting register and 13 qubits for work register (modular arithmetic), it cannot handle larger semiprimes without code modifications. The current gate count of 781 for 12-bit numbers indicates the circuit complexity.
For factoring larger numbers:

n-bit number requires ~2n counting qubits
Additional n+1 qubits needed for work register
Circuit depth grows with O(n³)

The implementation would need dynamic qubit allocation and optimized modular multiplication circuits to handle larger semiprimes efficiently.